# Les Finesses de Python

## Microclub 1er juin 2018

## Philippe Guglielmetti

---

## Pourquoi encore Python ?

> Il ne vaut pas la peine de connaître un langage qui ne modifie pas votre façon de penser la programmation.
> ([Alan Perlis](https://www.drgoulu.com/2008/01/21/perlisismes-les-dictons-informatiques-dalan-perlis/))

* #4 au [TIOBE](https://www.tiobe.com/tiobe-index/) (nombre de développeurs)
* #2 au [Madnight] (https://madnight.github.io/githut/#/issues/2018/1) (activité sur GitHub)
* #1 au [PYPL](http://pypl.github.io/PYPL.html) PopularitY of Programming Language (nombre de tutoriels suivis)



>Python est un langage de programmation objet, multi-paradigme ([wikipedia](https://fr.wikipedia.org/wiki/Python_(langage)))

= on peut programmer :
1. [comme une patate](http://entrian.com/goto/)
2. comme on en a l’habitude (classes, objets, ...)
3. comme on en a pas l’habitude
4. comme ~~un dieu~~ [Guido van Rossum](https://fr.wikipedia.org/wiki/Guido_van_Rossum)

---

* [Les Décorateurs](#deco)
  * [La Memoïsation](#memo)
  * [Le problème de l'arrêt](#stop)
    

<a id="deco"></a>
## Les Décorateurs


Un décorateur est une fonction qui modifie une fonction en l'enveloppant.
C'est un [patron de conception](https://fr.wikipedia.org/wiki/Patron_de_conception)

"Gang of four" (GOF) Erich Gamma, Richard Helm, Ralph Johnson, John Vlissides 
“Design Patterns - Catalogue de modèles de conceptions réutilisables” 
Vuibert, 1999, 490 p 

In [1]:
import logging # librairie standard

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [2]:
def fonction():
    """cette fonction logue son propre nom au niveau INFO, et sa doc au niveau DEBUG"""
    logging.info(fonction.__name__)
    logging.debug(fonction.__doc__)
    
fonction()

INFO:root:fonction


In [3]:
from Goulib import decorators

@decorators.debug
def fonction():
    """cette fonction logue son propre nom au niveau INFO, et sa doc au niveau DEBUG"""
    logging.info(fonction.__name__)
    logging.debug(fonction.__doc__)
    
fonction()

INFO:root:Entering fonction
INFO:root:fonction
DEBUG:root:cette fonction logue son propre nom au niveau INFO, et sa doc au niveau DEBUG
INFO:root:Exiting fonction


In [4]:
import functools # librairie standard

def timeit(func):
    @functools.wraps(func) # un décorateur qui simplifie l'écriture de décorateurs...
    def wrapper(*args, **kwds):
        import time
        t=time.time()
        f_result = func(*args, **kwds)
        t=time.time()-t
        params=', '.join(map(repr,args))
        logger.info('%s(%s) took %f seconds'%(func.__name__,params,t)) 
        return f_result
    return wrapper

In [5]:
def fib(n):
    """une très mauvaise implémentation de la suite de Fibonacci"""
    if n < 2:
      return 1
    return fib(n-1) + fib(n-2)

@timeit
def tfib(n): return fib(n)

[tfib(i) for i in range(0,40,5)]

INFO:root:tfib(0) took 0.000000 seconds
INFO:root:tfib(5) took 0.000000 seconds
INFO:root:tfib(10) took 0.000000 seconds
INFO:root:tfib(15) took 0.000000 seconds
INFO:root:tfib(20) took 0.002010 seconds
INFO:root:tfib(25) took 0.028006 seconds
INFO:root:tfib(30) took 0.269098 seconds
INFO:root:tfib(35) took 2.941747 seconds


[1, 8, 89, 987, 10946, 121393, 1346269, 14930352]

<a id="memo"></a>
### La [Memoïsation](https://fr.wikipedia.org/wiki/M%C3%A9mo%C3%AFsation)

Patron de conception qui accélère les appels répétés à une fonction en mémorisant les résultats en fonction des paramètres

In [8]:
def memoize(obj): # disponible dans Goulib.decorators
    cache = obj.cache = {} # simple dict. il existe des implémentations à mémoire limitée
    @functools.wraps(obj)
    def memoizer(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = obj(*args, **kwargs)
        return cache[key]
    return memoizer

In [9]:
@memoize
def mfib(n):
    """la même très mauvaise implémentation de la suite de Fibonacci"""
    if n < 2:
      return 1
    return mfib(n-1) + mfib(n-2)

@timeit
def tfib(n): return mfib(n)

[tfib(i) for i in range(0,40,5)]

INFO:root:tfib(0) took 0.000000 seconds
INFO:root:tfib(5) took 0.000000 seconds
INFO:root:tfib(10) took 0.000000 seconds
INFO:root:tfib(15) took 0.000000 seconds
INFO:root:tfib(20) took 0.000000 seconds
INFO:root:tfib(25) took 0.000000 seconds
INFO:root:tfib(30) took 0.000000 seconds
INFO:root:tfib(35) took 0.000000 seconds


[1, 8, 89, 987, 10946, 121393, 1346269, 14930352]

In [10]:
tfib(500)

INFO:root:tfib(500) took 0.026008 seconds


225591516161936330872512695036072072046011324913758190588638866418474627738686883405015987052796968498626

<a id="stop"></a>
### Le Timeout

In [35]:
i,n = 0,0
@decorators.timeout(t) # implémentation interrompant la thread
def fibloop():
    global i,n
    while True: # boucle infinie sans le décorateur ...
        i=i+1
        n=fib(i)
        
try:
    fibloop()
except decorators.TimeoutError:
    print('en %d secondes, on peut calculer %d termes'%(t,i))
    print('le dernier est',n)

en 3 secondes, on peut calculer 33 termes
le dernier est 3524578


In [11]:
#implémentation au niveau de l'itérateur de boucle
# dommage, pas de @itimeout for ... 
try:
    for i in decorators.itimeout(itertools.count(),t):
        n=mfib(i)
except decorators.TimeoutError:
    print('en %d secondes, on peut calculer %d termes'%(t,i))
    print('le dernier est',n)

en 3 secondes, on peut calculer 253189 termes
le dernier est ...


## Classes et objets

In [12]:
256+1 is 257

False

In [13]:
255+1 is 256

True

In [14]:
type(256)

int

In [15]:
type(int)

type

In [16]:
type(type(int))

type

In [17]:
def f(n):
    """factorielle récursive"""
    return 1 if n<2 else n*f(n-1)
type(f)

function

In [18]:
type(type(f))

type

In [19]:
type(type(int)) is type(type(f))

True

## Introspection

In [20]:
print(dir(256)) # membres de l'objet

['__abs__', '__add__', '__and__', '__bool__', '__ceil__', '__class__', '__delattr__', '__dir__', '__divmod__', '__doc__', '__eq__', '__float__', '__floor__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getnewargs__', '__gt__', '__hash__', '__index__', '__init__', '__init_subclass__', '__int__', '__invert__', '__le__', '__lshift__', '__lt__', '__mod__', '__mul__', '__ne__', '__neg__', '__new__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rlshift__', '__rmod__', '__rmul__', '__ror__', '__round__', '__rpow__', '__rrshift__', '__rshift__', '__rsub__', '__rtruediv__', '__rxor__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__trunc__', '__xor__', 'bit_length', 'conjugate', 'denominator', 'from_bytes', 'imag', 'numerator', 'real', 'to_bytes']


In [21]:
print(dir(f)) # membres de l'objet

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [22]:
f.__doc__

'factorielle récursive'

In [23]:
import inspect # https://docs.python.org/3/library/inspect.html
inspect.getsourcelines(f)

(['def f(n):\n',
  '    """factorielle récursive"""\n',
  '    return 1 if n<2 else n*f(n-1)\n'],
 1)

## programmation fonctionnelle

In [24]:
for i in range(10):
    print(f(i), end=" ")

1 1 2 6 24 120 720 5040 40320 362880 

In [25]:
r=range(10)
print(r)
print(_ for _ in r)
print([_ for _ in r])

range(0, 10)
<generator object <genexpr> at 0x0000022CBFFFAA40>
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [26]:
res=list(map(f,r))
res

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880]

In [27]:
import operator
from functools import reduce # était prédéfini en python 2.7
reduce(operator.add,res)


409114

## [Librairies](https://docs.python.org/3/library/)
quelques librairies standart fort utiles:

### [array](https://docs.python.org/3/library/array.html)
tableaux (comme des listes, mais type fixe)

In [28]:
import sys
from array import array
l=list(range(1000))
print(sys.getsizeof(l),sys.getsizeof(array('i',l)))

9112 4064


### [re](https://docs.python.org/3/library/re.html)
regular expressions